In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
# %config Completer.use_jedi = False

In [242]:
import os
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
import pandas as pd
import pickle
import json
from itertools import repeat
import multiprocess
from pprint import pprint
import numpy as np
import pvlib
import proplot as pplt
import ipv_workbench as ipv
from ipv_workbench.utilities import utils, circuits, time_utils
from ipv_workbench.devices import devices
from ipv_workbench.simulator import simulations as ipv_sim
from ipv_workbench.simulator import simulations_mp as ipv_sim_mp
from ipv_workbench.simulator import calculations as ipv_calc
from ipv_workbench.simulator import effective_irradiance as eff_irrad
from ipv_workbench.visualize import plots as ipv_plots
from ipv_workbench.translators import panelizer
from ipv_workbench.sample import dummies
from tqdm import tqdm
import functools, operator, collections
import multiprocess as mp
import time
import csv
import datetime

In [27]:
def read_headers(file_content):
        d={}
        csvreader = csv.reader(file_content, delimiter=',', quotechar='"')
        for row in csvreader:
            if row[0].isdigit():
                break
            else:
                d[row[0]]=row[1:]

        return d
    

    

In [80]:
epw_file = "/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/polikseni_share/epw_morph/fluntern_TMYx2007-2021.epw"
tmy_location = utils.tmy_location(epw_file)
tmy = utils.tmy_to_dataframe(epw_file)


In [219]:
sp_file = "/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/_noshare/B1391/surface_{1391;0;3}/radiance_results/annual_irradiance/model/grid/radiancegrid_139103_8514sensors.pts"
sp_file2 = "/Users/jmccarty/Desktop/Bano_Project/radiance_results_no_accelerad_tree_not_modified/annual_irradiance/model/grid/radiancegrid_139103_8514sensors.pts"
total_results = "/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/_noshare/B1391/surface_{1391;0;3}/radiance_results/annual_irradiance/results/total/radiancegrid_139103_8514sensors.ill"
total_results2 = "/Users/jmccarty/Desktop/Bano_Project/radiance_results_no_accelerad_tree_not_modified/annual_irradiance/results/total/radiancegrid_139103_8514sensors.ill"
total_results_ghp = "/Users/jmccarty/Desktop/Bano_Project/testing_negative/annual_irradiance/results/total/radiancegrid_1391010_15900sensors.ill"
dir_results = "/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/_noshare/B1391/surface_{1391;0;3}/radiance_results/annual_irradiance/results/direct/radiancegrid_139103_8514sensors.ill"
dir_results2 = "/Users/jmccarty/Desktop/Bano_Project/radiance_results_no_accelerad_tree_not_modified/annual_irradiance/results/direct/radiancegrid_139103_8514sensors.ill"
dir_results_ghp = "/Users/jmccarty/Desktop/Bano_Project/testing_negative/annual_irradiance/results/direct/radiancegrid_1391010_15900sensors.ill"

sun_up = "/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/_noshare/B1391/surface_{1391;0;3}/radiance_results/annual_irradiance/results/total/sun-up-hours.txt"

In [230]:
sp_data = aoi.load_sensor_points(sp_file)
sp_data.loc[4592]

x       134.600547
y       155.857732
z        23.103509
xdir     -0.560424
ydir     -0.828206
zdir      0.000000
Name: 4592, dtype: float64

In [222]:
# load in data
irrad_total_df = utils.build_full_ill(sun_up, total_results2)
irrad_direct_df = utils.build_full_ill(sun_up, dir_results2)

irrad_diffuse_df = irrad_total_df - irrad_direct_df
sensors_df = aoi.load_sensor_points(sp_file)
irrad_diffuse_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [204]:
irrad = pd.read_csv(total_results2, delimiter=' ', header=None)#.iloc[:, 1:].T.reset_index(drop=True)

In [226]:
# irrad_total_df[6704].sort_values()

# def does_it_have_negatives(result_file):
#     irrad_total_df = utils.build_full_ill(sun_up, result_file)
#     if irrad_total_df.min(axis=1).sort_values().values[0] < 0:
#         print(f"{result_file}")



-80.407898

In [141]:
# calculate AOI mod
vector = np.array([sensors_df.at[0, 'xdir'], 
                   sensors_df.at[0, 'ydir'],
                   sensors_df.at[0, 'zdir']])

surface_azimuth, surface_tilt = aoi.vector_to_tilt_and_azimuth(vector)

surface_tilt_deg = np.rad2deg(surface_tilt)
surface_azimuth_deg = np.rad2deg(surface_azimuth)
solar_zenith_deg = solar_position_hoy['apparent_zenith'].values
solar_azimuth_deg = solar_position_hoy['azimuth'].values

aoi_mod = pvlib.irradiance.aoi(surface_tilt_deg,surface_azimuth_deg,solar_zenith_deg,solar_azimuth_deg)

In [142]:
# get solar position data
hoy = 3998

solar_position_hoy = pvlib.solarposition.get_solarposition(time_utils.hoy_to_date(hoy), 
                                      tmy_location['lat'], 
                                      tmy_location['lon'], 
                                      altitude=tmy_location['elevation'], 
                                      pressure=tmy.loc[hoy]['atmos_Pa'], 
                                      method='nrel_numpy', 
                                      temperature=tmy.loc[hoy]['drybulb_C']
                                     )
"""
* apparent_elevation : apparent sun elevation accounting for
          atmospheric refraction.
* elevation : actual elevation (not accounting for refraction)
  of the sun in decimal degrees, 0 = on horizon.
  The complement of the zenith angle.
* azimuth : Azimuth of the sun in decimal degrees East of North.
  This is the complement of the apparent zenith angle.
* apparent_zenith : apparent sun zenith accounting for atmospheric
  refraction. (this si the one we need *solar_zenith must be >=0 and <=180)
* zenith : Solar zenith angle
* solar_time : Solar time in decimal hours (solar noon is 12.00).
"""


'\n* apparent_elevation : apparent sun elevation accounting for\n          atmospheric refraction.\n* elevation : actual elevation (not accounting for refraction)\n  of the sun in decimal degrees, 0 = on horizon.\n  The complement of the zenith angle.\n* azimuth : Azimuth of the sun in decimal degrees East of North.\n  This is the complement of the apparent zenith angle.\n* apparent_zenith : apparent sun zenith accounting for atmospheric\n  refraction. (this si the one we need *solar_zenith must be >=0 and <=180)\n* zenith : Solar zenith angle\n* solar_time : Solar time in decimal hours (solar noon is 12.00).\n'

In [161]:
_sp = 0

e_dir = irrad_direct_df[_sp].loc[hoy]
e_diff = irrad_diffuse_df[_sp].loc[hoy]
theta = np.deg2rad(aoi_mod)
aa_r = 0.17

aoi.e_effective(e_dir,e_diff,theta,aa_r)

ALERT
e_dir is  17.14404
e_diff is  -17.39994502
e_eff is  [-9.73147425]
k1 is  [0.39639275]


0

In [159]:
aoi.k_martin_ruiz(np.deg2rad(60), aa_r)

0.9498448031311924

In [232]:
tmy_og = utils.tmy_to_dataframe("/Users/jmccarty/Nextcloud/Teaching/22_HS/polikseni_bano/polikseni_share/epw_morph/fluntern_TMYx2007-2021.epw")

In [251]:
eff_irrad.front_cover_loss(1000,"pure_white")


array([548.])